In [29]:
import os
import pandas as pd

input_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
output_path = 'onetask_5_labels/all_label.csv'
df = pd.read_csv(input_path)
chosen = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL']
mapping = {'NILM' :0, 'ASC-US' :1, 'LSIL' :2, 'ASC-H' :3, 'HSIL' :4}
df = df[df['wsi_label'].isin(chosen)]
output_df = df[['wsi_id', 'wsi_label']].copy()
output_df['wsi_id'] = output_df['wsi_id'].str.replace('.pt', '')
output_df['wsi_label'] = output_df['wsi_label'].map(mapping)
output_df.to_csv(output_path, index=False)

In [1]:
import os
import pandas as pd
import numpy as np

np.random.seed(0)
label_dir = '/data/wsi/TCTGC10k-labels/9_labels'
input_path = 'all_label.csv'
input_path = os.path.join(label_dir, input_path)
output_path = ['train_label.csv', 'val_label.csv', 'test_label.csv']
output_path = [os.path.join(label_dir, p) for p in output_path]
ratios = [0.8, 0.1, 0.1]
df = pd.read_csv(input_path)
df_shuffle = df.sample(frac=1).reset_index(drop=True)
df_split = [df_shuffle.iloc[:int(len(df_shuffle) * ratios[0])],
            df_shuffle.iloc[int(len(df_shuffle) * ratios[0]):int(len(df_shuffle) * (ratios[0] + ratios[1]))],
            df_shuffle.iloc[int(len(df_shuffle) * (ratios[0] + ratios[1])):]]
for i in range(3):
    df_split[i].to_csv(output_path[i], index=False, header=False)


In [31]:
import os
import pandas as pd
label_dir = 'onetask_5_labels'
label_path = ['train_label.csv', 'val_label.csv', 'test_label.csv']
for i in range(3):
    label_path[i] = os.path.join(label_dir, label_path[i])
    df = pd.read_csv(label_path[i])
    print(df.iloc[:,1].value_counts())
    print('######################{}'.format(len(df)))


0
0    4723
1     983
2     701
3     234
4     198
Name: count, dtype: int64
######################6839
1
0    579
1    118
2    106
3     26
4     25
Name: count, dtype: int64
######################854
0
0    585
1    124
2     94
3     33
4     18
Name: count, dtype: int64
######################854


# 将原始文件抽取需要的信息(wsi_name, wsi_id)到all_label.csv

In [5]:
import pandas as pd
import os

output_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/gc10k/9_labels'
label_path = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df = pd.read_csv(label_path)
mapping = {'NILM' :0, 'ASC-US' :1, 'LSIL' :2, 'ASC-H' :3, 'HSIL' :4,\
            'AGC':5, 'BV':6, 'M':7, 'T':8}
# id2class = {v, k for k, v in mapping.items()}
output_file = 'all_label.csv'
df['wsi_label'] = df['wsi_label'].map(mapping)
df['wsi_name'] = df['wsi_name'].str.replace('.pt', '')
df_output = df[['wsi_name', 'wsi_label']].copy()
df_output.to_csv(os.path.join(output_path, output_file), index=False, header=False)

# 根据all_label.csv进行数据集的划分

In [5]:
import pandas as pd
import os

label_dir = '/data/wsi/TCTGC10k-labels/6_labels'
all_label_path = os.path.join(label_dir, 'TCTGC10k-v15-all.csv')
df = pd.read_csv(all_label_path)
print(len(df))
df_shuffled = df.sample(frac=1, random_state=42)

# 根据9:1比例划分
train_ratio = 0.8
df_train = df_shuffled.iloc[:int(len(df_shuffled) * train_ratio)]
df_test = df_shuffled.iloc[int(len(df_shuffled) * train_ratio):]

df_train.to_csv(os.path.join(label_dir, 'TCTGC10k-v15-train.csv'), index=False)
df_test.to_csv(os.path.join(label_dir, 'TCTGC10k-v15-test.csv'), index=False)

print('df_train')
print(df_train['wsi_label'].value_counts())
print('df_test')
print(df_test['wsi_label'].value_counts())

8476
df_train
wsi_label
nilm     4714
ascus     881
lsil      689
asch      228
hsil      185
agc        83
Name: count, dtype: int64
df_test
wsi_label
nilm     1171
ascus     228
lsil      174
asch       58
hsil       54
agc        11
Name: count, dtype: int64


# 挑选出Cancer标注的WSI

In [2]:
import os
import pandas as pd

root_dir = '/data/wsi/TCTGC10k-labels'
input_dir = os.path.join(root_dir, '9_labels', 'TCTGC10k-v15-all.csv')
output_dir = os.path.join(root_dir, '6_labels', 'TCTGC10k-v15-all.csv')
chosen_labels = ['nilm', 'ascus', 'asch', 'lsil', 'hsil', 'agc']
df = pd.read_csv(input_dir)
# 过滤并保存
filtered_df = df[df['wsi_label'].isin(chosen_labels)]
filtered_df.to_csv(output_dir, index=False)
filtered_df['wsi_label'].value_counts()

8476